Temperature animation for Run 08 (sketches for python scripts)

In [14]:
#KRM

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.colors as mcolors

from math import *

import scipy.io

import scipy as spy

#%matplotlib inline

from netCDF4 import Dataset

import pylab as pl




import matplotlib.animation as animation


In [15]:
#'''
#NAME
#    Custom Colormaps for Matplotlib
#PURPOSE
#    This program shows how to implement make_cmap which is a function that
#    generates a colorbar.  If you want to look at different color schemes,
#    check out https://kuler.adobe.com/create.
#PROGRAMMER(S)
#    Chris Slocum
#REVISION HISTORY
#    20130411 -- Initial version created
#    20140313 -- Small changes made and code posted online
#    20140320 -- Added the ability to set the position of each color
#'''

def make_cmap(colors, position=None, bit=False):
    #'''
    #make_cmap takes a list of tuples which contain RGB values. The RGB
    #values may either be in 8-bit [0 to 255] (in which bit must be set to
    #rue when called) or arithmetic [0 to 1] (default). make_cmap returns
    #a cmap with equally spaced colors.
    #Arrange your tuples so that the first color is the lowest value for the
    #colorbar and the last is the highest.
    #position contains values from 0 to 1 to dictate the location of each color.
    #'''
    import matplotlib as mpl
    import numpy as np
    bit_rgb = np.linspace(0,1,256)
    if position == None:
        position = np.linspace(0,1,len(colors))
    else:
        if len(position) != len(colors):
            sys.exit("position length must be the same as colors")
        elif position[0] != 0 or position[-1] != 1:
            sys.exit("position must start with 0 and end with 1")
    if bit:
        for i in range(len(colors)):
            colors[i] = (bit_rgb[colors[i][0]],
                         bit_rgb[colors[i][1]],
                         bit_rgb[colors[i][2]])
    cdict = {'red':[], 'green':[], 'blue':[]}
    for pos, color in zip(position, colors):
        cdict['red'].append((pos, color[0], color[0]))
        cdict['green'].append((pos, color[1], color[1]))
        cdict['blue'].append((pos, color[2], color[2]))

    cmap = mpl.colors.LinearSegmentedColormap('my_colormap',cdict,256)
    return cmap

In [16]:
# Get field from MITgcm netCDF output
#
''' :statefile : string with /path/to/state.0000000000.t001.nc
    :fieldname : string with the variable name as written on the netCDF file ('Temp', 'S','Eta', etc.)'''

def getField(statefile, fieldname):
    
    StateOut = Dataset(statefile)
    
    Fld = StateOut.variables[fieldname][:]
    
    shFld = np.shape(Fld)
    
        
    if len(shFld) == 2:
        
        Fld2 = np.reshape(Fld,(shFld[0],shFld[1])) # reshape to pcolor order
        return Fld2 
    
    elif len(shFld) == 3:
        
        Fld2 = np.zeros((shFld[0],shFld[1],shFld[2])) 
        Fld2 = np.reshape(Fld,(shFld[0],shFld[1],shFld[2])) # reshape to pcolor order
        return Fld2
        
    elif len(shFld) == 4:
        
        Fld2 = np.zeros((shFld[0],shFld[1],shFld[2],shFld[3])) 
        Fld2 = np.reshape(Fld,(shFld[0],shFld[1],shFld[2],shFld[3])) # reshape to pcolor order
        return Fld2
        
    else:
        
        print (' Check size of field ')
    

In [55]:
filenameb='/ocean/kramosmu/MITgcm/CanyonUpwelling/180x180x35_BodyForcing_6Tr_LinProfiles/run08/output_0001/state.0000000000.t001.nc'
StateOutb = Dataset(filenameb)

#for dimobj in StateOut.variables.values():
#   print dimobj


filename2b='/ocean/kramosmu/MITgcm/CanyonUpwelling/180x180x35_BodyForcing_6Tr_LinProfiles/run08/output_0001/grid.t001.nc'
GridOutb = Dataset(filename2b)

for dimobj in GridOutb.variables.values():
    print dimobj


filename3b='/ocean/kramosmu/MITgcm/CanyonUpwelling/180x180x35_BodyForcing_6Tr_LinProfiles/run08/output_0001/ptracers.0000000000.t001.nc'
PtracersOutb = Dataset(filename3b)

#for dimobj in PtracersOut.variables.values():
#    print dimobj


<type 'netCDF4.Variable'>
float64 Z(Z)
    long_name: vertical coordinate of cell center
    units: meters
    positive: up
unlimited dimensions: 
current shape = (35,)

<type 'netCDF4.Variable'>
float64 RC(Z)
    description: R coordinate of cell center
    units: m
unlimited dimensions: 
current shape = (35,)

<type 'netCDF4.Variable'>
float64 Zp1(Zp1)
    long_name: vertical coordinate of cell interface
    units: meters
    positive: up
unlimited dimensions: 
current shape = (36,)

<type 'netCDF4.Variable'>
float64 RF(Zp1)
    description: R coordinate of cell interface
    units: m
unlimited dimensions: 
current shape = (36,)

<type 'netCDF4.Variable'>
float64 Zu(Zu)
    long_name: vertical coordinate of lower cell interface
    units: meters
    positive: up
unlimited dimensions: 
current shape = (35,)

<type 'netCDF4.Variable'>
float64 RU(Zu)
    description: R coordinate of upper interface
    units: m
unlimited dimensions: 
current shape = (35,)

<type 'netCDF4.Variable'>
floa

In [18]:
# General input

nx = 180
ny = 180
nz = 35
nta = 21 # t dimension size run 04 and 05 (output every 2 hr for 4.5 days)
ntc = 21 # t dimension size run 06 (output every half-day for 4.5 days)

z = StateOutb.variables['Z']
#print(z[:])
Time = StateOutb.variables['T']
print(Time[:])

xc = getField(filename2b, 'XC') # x coords tracer cells

yc = getField(filename2b, 'YC') # y coords tracer cells


[      0.   43200.   86400.  129600.  172800.  216000.  259200.  302400.
  345600.  388800.  432000.  475200.  518400.  561600.  604800.  648000.
  691200.  734400.  777600.  820800.  864000.]


In [19]:
zlev =  31             # level 14 corresponds to 162.5 m , near shelf break

timesc = [1,3,5,10,15,20]    # These correspond to 1,2,4,6,8,10 days



In [20]:
ugridb = getField(filenameb,'U')
vgridb = getField(filenameb,'V')


print(np.shape(ugridb))


print(np.shape(vgridb))



(21, 35, 180, 181)
(21, 35, 181, 180)


In [21]:
tempb = getField(filenameb, 'Temp')


temp0b = np.ma.masked_values(tempb, 0)



MASKb = np.ma.getmask(temp0b)


In [51]:
plt.rcParams.update({'font.size':13})


 
NumLev = 30 # number of levels for contour


In [52]:
####  PLOT ####



def animateTemp(tt):
    """Generate frames for Temperature animation Run10 in 180x180x35_BodyForcing_6Tr_LinProfiles
    tt corresponds to the time output
    """
    
    kk=1
    plt.cla
    
            
    ### Temperature 
            
    ax1 = fig.add_subplot(2, 2, 1)
    ax1.set_aspect(0.75)
    plt.contourf(xc,yc,temp0b[tt,4,:,:],NumLev,cmap='gist_heat')
            
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            
    plt.xlabel('m')
    plt.ylabel('m')
            
            
    plt.title(" depth=%1.1f m,%1.1f days " % (z[4],tt/2.))
            
    ax2 = fig.add_subplot(2, 2, 2)
    ax2.set_aspect(0.75)
    plt.contourf(xc,yc,temp0b[tt,14,:,:],NumLev,cmap='gist_heat')
            
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            
    plt.xlabel('m')
    plt.ylabel('m')
            
            
    plt.title(" depth=%1.1f m,%1.1f days " % (z[14],tt/2.))
            
    ax3 = fig.add_subplot(2, 2, 3)
    ax3.set_aspect(0.75)
    plt.contourf(xc,yc,temp0b[tt,24,:,:],NumLev,cmap='gist_heat')
            
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            
    plt.xlabel('m')
    plt.ylabel('m')
            
            
    plt.title(" depth=%1.1f m,%1.1f days " % (z[24],tt/2.))
            
    ax4 = fig.add_subplot(2, 2, 4)
    ax4.set_aspect(0.75)
    plt.contourf(xc,yc,temp0b[tt,31,:,:],NumLev,cmap='gist_heat')
            
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            
    plt.xlabel('m')
    plt.ylabel('m')
            
            
    plt.title(" depth=%1.1f m,%1.1f days " % (z[31],tt/2.))
            
            
    ax1.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax2.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax3.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax4.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    
    
            
           
            

In [53]:

plt.rcParams.update({'font.size':14})

colorsTemp = [(245.0/255.0,245/255.0,245./255.0), (255/255.0,20/255.0,0)] #(khaki 1246/255.0,143./255.0  ,orangered2)
posTemp = [0, 1] 

fig= plt.figure(figsize=(12,9))



    

#The animation function    
anim = animation.FuncAnimation(fig, animateTemp, frames=21)

#cb = plt.colorbar(pad = 0.5)
#cb.set_label(r'$^{\circ}$C',position=(1, 0),rotation=0)

plt.show()
       
#A line that makes it all work
#mywriter = animation.FFMpegWriter()

#Save in current folder
#anim.save('Run08Temp.mp4',writer=mywriter)


